In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE
import copy
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = "{:,.4f}".format
sm = SMOTE(random_state=42)

testset_frac = 0.25
data_samples = 80000 

df_full = pd.read_csv('data/Encoded.csv')
df_full = df_full.sample(n=data_samples)
df_full = df_full.iloc[:, 1:]
df_full = df_full.drop(columns=['Attack Tool', 'Label', 'sVid', 'dVid', '54'])
df_full = df_full.fillna(df_full.median())

<ipython-input-1-0c734b6eda91>:45: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_full = df_full.fillna(df_full.median())


In [2]:
learning_rate = 0.01
numEpoch = 20
batch_size = 32
momentum = 0.9
print_amount=3
number_of_slices = 1
isSmote = False
runtime = 21

file_name = "federated_" + str(isSmote) + "_" + str(number_of_slices)  + "_" + str(runtime) + ".txt"
file = open(file_name, "w")

In [3]:
dummies = pd.get_dummies(df_full['Attack Type'])  # Classification
outcomes = dummies.columns
print(outcomes)
num_classes = len(outcomes)
Y = dummies.values
print(df_full['Attack Type'].value_counts())
print(num_classes)

Index(['Benign', 'HTTPFlood', 'ICMPFlood', 'SYNFlood', 'SYNScan',
       'SlowrateDoS', 'TCPConnectScan', 'UDPFlood', 'UDPScan'],
      dtype='object')
Benign            31406
UDPFlood          30087
HTTPFlood          9299
SlowrateDoS        4770
SYNScan            1323
TCPConnectScan     1290
UDPScan            1079
SYNFlood            668
ICMPFlood            78
Name: Attack Type, dtype: int64
9


In [4]:
df_Benign = df_full[df_full['Attack Type']=='Benign']
df_HTTPFlood= df_full[df_full['Attack Type']=='HTTPFlood']
df_UDPFlood= df_full[df_full['Attack Type']=='UDPFlood']
df_SYNFlood= df_full[df_full['Attack Type']=='SYNScan']
df_SlowrateDoS= df_full[df_full['Attack Type']=='SlowrateDoS']

df_Benign.loc[(df_Benign['Attack Type'] == 'Benign'), 'Attack Type'] = 0
df_HTTPFlood.loc[(df_HTTPFlood['Attack Type'] == 'HTTPFlood'), 'Attack Type'] = 2
df_UDPFlood.loc[(df_UDPFlood['Attack Type'] == 'UDPFlood'), 'Attack Type'] = 1
df_SYNFlood.loc[(df_SYNFlood['Attack Type'] == 'SYNScan'), 'Attack Type'] = 4
df_SlowrateDoS.loc[(df_SlowrateDoS['Attack Type'] == 'SlowrateDoS'), 'Attack Type'] = 3

print(df_Benign.shape)
print(df_HTTPFlood.shape)
print(df_UDPFlood.shape)
print(df_SYNFlood.shape)
print(df_SlowrateDoS.shape)

df_filterd = pd.concat([df_Benign,df_HTTPFlood,df_UDPFlood,df_SYNFlood,df_SlowrateDoS])
print(df_filterd['Attack Type'].value_counts())
print(df_filterd.shape)
type_df = df_filterd['Attack Type'].copy()
data_df = df_filterd.drop('Attack Type',axis=1)
print(type_df.shape)


data_df = data_df / data_df.max()
df_normalized = pd.concat([data_df,type_df],axis=1)

(31406, 90)
(9299, 90)
(30087, 90)
(1323, 90)
(4770, 90)
0    31406
1    30087
2     9299
3     4770
4     1323
Name: Attack Type, dtype: int64
(76885, 90)
(76885,)


In [5]:
def divide_train_test(df, propotion=0.1):
    
    df_train = []
    df_test = []
    for key,val in df['Attack Type'].value_counts().iteritems():
        df_part = df[df['Attack Type'] == key]
        df_test.append(df_part[0: int(df_part.shape[0]*propotion)])
        df_train.append(df_part[int(df_part.shape[0]*propotion):df_part.shape[0]])
        
    return df_train,df_test

In [6]:
df_train, df_test = divide_train_test(df_normalized,propotion=0.1)


df_train = pd.concat(df_train)
print(df_train.shape)
df_train=df_train.dropna(axis=1)

# df_train = df_train.drop(df_train.loc[df_train['Attack Type']==1].index, inplace=True)
# df_train = df_train.loc[df_train['Attack Type']!=1]
df_train = df_train.loc[df_train['Attack Type']!=1]
df_train = df_train.loc[df_train['Attack Type']!=2]
df_train = df_train.loc[df_train['Attack Type']!=3]
print(df_train.shape)


df_test = pd.concat(df_test)
df_test=df_test.dropna(axis=1)


y_test = df_test.pop('Attack Type').values
x_test = df_test.values

print('Test set size is : x => ' + str(x_test.shape) + ' y => ' + str(y_test.shape))
x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test.astype('int')).type(torch.LongTensor)

y_info = df_train.pop('Attack Type').values
# print(np.isnan(y_info))
x_info = df_train.values

y_info = y_info.astype('int')
x_train = torch.tensor(x_info).float()
y_train = torch.tensor(y_info).type(torch.LongTensor)        

print(x_train.shape[1])
inputs = x_test.shape[1]
outputs = 5
print(inputs,outputs)

(69199, 90)
(29457, 85)
Test set size is : x => (7686, 84) y => (7686,)
84
84 5


In [7]:
class Net2nn(nn.Module):
    def __init__(self, inputs, outputs):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(inputs,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,outputs)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [8]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
#         print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [9]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [10]:
def confusion_mat(model, test_loader):
    y_pred = []
    y_true = []

    # iterate over test data
    for inputs, labels in test_loader:
        output = model(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)
    precisionv = precision_score(y_true,y_pred,average='macro')
    recallv = recall_score(y_true,y_pred,average='macro')
    print('precision value: '+str(precisionv))
    print('recall value: '+ str(recallv))
#     df_cm = pd.DataFrame(cf_matrix, index=[i for i in Counter(y_test)],
#                          columns=[i for i in Counter(y_test)])
#     plt.figure(1)
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.figure(figsize=(12, 7))

#     sn.heatmap(df_cm, annot=True).set(xlabel='Predicted label', ylabel='True label')
#     plt.savefig('D:\\learning\\PyTorch\\experiment\\cf\\cf_fl_'+str(self.number_of_slices)+'.png')
    print('confusion matrix for normal scenario for slices : ' + str(number_of_slices))
    print(cf_matrix)
    file.write('\ncf matrix for slice :' + str(number_of_slices))
    file.write('\n'+str(cf_matrix))

In [11]:
# initial_model = Net2nn()
# initial_optimizer = torch.optim.SGD(initial_model.parameters(), lr=0.01, momentum=0.9)
# initial_criterion = nn.CrossEntropyLoss()

train_acc = []
test_acc = []
train_loss = []
test_loss = []

centralized_model = Net2nn(inputs, outputs)
centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
centralized_criterion = nn.CrossEntropyLoss()
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model, train_dl, centralized_criterion, centralized_optimizer)
    central_test_loss, central_test_accuracy = validation(centralized_model, test_dl, centralized_criterion)

    train_acc.append(central_train_accuracy)
    train_loss.append(central_train_loss)
    test_acc.append(central_test_accuracy)
    test_loss.append(central_test_loss)
    print(central_test_accuracy)
#     print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))


0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864
0.4257090814467864


In [12]:
confusion_mat(centralized_model, test_dl)

precision value: 0.2831347630394493
recall value: 0.4
confusion matrix for normal scenario for slices : 1
[[3140    0    0    0    0]
 [3008    0    0    0    0]
 [ 929    0    0    0    0]
 [ 477    0    0    0    0]
 [   0    0    0    0  132]]


D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
